In [2]:
import folium
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from geojson import Feature, Point, FeatureCollection
import shapely
import json
from tqdm import tqdm
import psycopg2 as pg

'import psycopg2 as pg\n'

In [3]:
def get_masterplan(schools, buildings, polygon: json, add_building: json, delet_building: list[int]) -> json:

    """The function calculates the indicators of the master plan for a certain territory.

    :param polygon: the territory within which indicators will be calculated in GeoJSON format.
    :param add_building: the building that will be added to the territory in GeoJSON format.
    :param delet_building: the building that will be deleted from the territory in List format.

    city_model = CityInformationModel(city_name="saint-petersburg", city_crs=32636)

    :example: Masterplan(city_model).get_masterplan(polygon, add_building, delet_building=[1, 2, 3])

    :return: dictionary with the name of the indicator and its value in JSON format.
    """
    buildings = gpd.GeoDataFrame.from_features(buildings).set_crs(4326).to_crs(32636)
    polygon = gpd.GeoDataFrame.from_features([polygon]).set_crs(4326).to_crs(32636)
    land_with_buildings = gpd.sjoin(buildings, polygon, how="inner")
    buffer = polygon.buffer(1600)
    schools = schools.to_crs(32636)
    filtered_schools = schools[schools.within(buffer)]

    if add_building is not None:
        add_building = gpd.GeoDataFrame.from_features(add_building).set_crs(4326).to_crs(32636)
        land_with_buildings = land_with_buildings.append(add_building)

    if delet_building is not None:
        delet_building = pd.DataFrame(delet_building)
        delet_building.columns = ["functional_object_id"]
        land_with_buildings = land_with_buildings[
            ~land_with_buildings["functional_object_id"].isin(delet_building["functional_object_id"])
        ]

    land_with_buildings_living = land_with_buildings[land_with_buildings["is_living"] == True]

    hectare = 10000
    living = 80
    commerce = 20

    land_area = polygon.area / hectare
    land_area = land_area.squeeze()
    land_area = np.around(land_area, decimals=2)

    buildings_area = land_with_buildings["basement_area"].sum()

    dev_land_procent = ((buildings_area / hectare) / land_area) * 100
    dev_land_procent = np.around(dev_land_procent, decimals=2)

    dev_land_area = land_with_buildings["basement_area"] * land_with_buildings["storeys_count"]
    dev_land_area = dev_land_area.sum() / hectare
    dev_land_area = np.around(dev_land_area, decimals=2)

    dev_land_density = dev_land_area / land_area
    dev_land_density = np.around(dev_land_density, decimals=2)

    land_living_area = land_with_buildings_living["basement_area"] * land_with_buildings_living["storeys_count"]
    land_living_area = (land_living_area.sum() / hectare) / 100 * living
    land_living_area = np.around(land_living_area, decimals=2)

    dev_living_density = land_living_area / land_area
    dev_living_density = np.around(dev_living_density, decimals=2)

    population = land_with_buildings["population"].sum()
    population = population.astype(int)

    population_density = population / land_area.squeeze()
    population_density = np.around(population_density, decimals=2)

    living_area_provision = (land_living_area * hectare) / population
    living_area_provision = np.around(living_area_provision, decimals=2)

    land_business_area = (land_living_area / living) * commerce
    land_business_area = np.around(land_business_area, decimals=2)

    building_height_mode = land_with_buildings["storeys_count"].mode().squeeze()
    building_height_mode = building_height_mode.astype(int)

    free_seats = filtered_schools["capacity"].sum()

    data = [
        land_area,
        dev_land_procent,
        dev_land_area,
        dev_land_density,
        land_living_area,
        dev_living_density,
        population,
        population_density,
        living_area_provision,
        land_business_area,
        building_height_mode,
        free_seats,
    ]
    index = [
        "land_area",
        "dev_land_procent",
        "dev_land_area",
        "dev_land_density",
        "land_living_area",
        "dev_living_density",
        "population",
        "population_density",
        "living_area_provision",
        "land_business_area",
        "building_height_mode",
        "free_seats",
    ]
    df_indicators = pd.Series(data, index=index)

    return json.loads(df_indicators.to_json())

In [4]:
polygon = {
    "type": "Feature",
    "geometry": {
        "coordinates": [
            [
                [30.28236360437208, 59.94259995775951],
                [30.274335607213317, 59.94683560022031],
                [30.262799655918656, 59.943476595267526],
                [30.261458266233234, 59.93769831060115],
                [30.273799051338784, 59.934472783622056],
                [30.28935917168954, 59.934472783622056],
                [30.2982123436135, 59.939848487642536],
                [30.294456452493648, 59.94307349180241],
                [30.28236360437208, 59.94259995775951],
            ]
        ],
        "type": "Polygon",
    },
    "properties": {"name": "Василеостровская", "cities_id": 1, "description": "Моя любимая станция"},
    "id": "524a0867fabe66a657e220e50549e604",
}

In [5]:
buildings = gpd.read_file("building_spb.geojson")
schools = gpd.read_file("schools_spb.geojson")

In [ ]:
polygon = gpd.GeoDataFrame.from_features([polygon]).set_crs(4326).to_crs(32636)
buffer = polygon.buffer(1600)
schools = schools.to_crs(32636)
filtered_schools = schools[schools.within(buffer)]

/Users/mvin/Desktop/CityGeoTools/CityGeoTools/venv/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [ ]:
indicators = get_masterplan(
    schools=schools,
    buildings=buildings,
    polygon=polygon,
    add_building=None,
    delet_building=None,
)
indicators